In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [7]:


import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
def extract_feature(file_name,mfcc,chroma,mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X= sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        result=np.array([])
        if chroma:
            stft=np.abs(librosa.stft(X))
            chroma_feat=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
            result=np.hstack((result,chroma_feat))
       
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
            result=np.hstack((result,mfccs))
        if mel:
            mel_spec=np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
            result=np.hstack((result,mel_spec))
    return result


In [4]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised',
}
observed_emotions=['calm','happy','fearful','disgust']

In [8]:
def my_dataset(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("speech-recc\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x),y,test_size=test_size,random_state=8)


In [9]:
x_train,x_test,y_train,y_test=my_dataset(test_size=0.25)

In [23]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=850)

In [24]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=850)

In [25]:
y_pred=model.predict(x_test)
print(x_test.shape)

(192, 180)


In [26]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print(accuracy)

0.6979166666666666


In [27]:
import joblib
joblib.dump(model,'sentiment_model.pkl')

['sentiment_model.pkl']

In [28]:
model=joblib.load('sentiment_model.pkl')
file=os.path.basename("C:\\Users\\Jaanhavi And Malhar\\mycodes\\projects\\speech_rec\\test.wav")
feature=extract_feature(file,mfcc=True,chroma=True,mel=True)

y=model.predict(feature.reshape(1,-1))
